In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import GPy
%matplotlib inline
plt.style.use('ggplot')

In [31]:
def gauss_gp(y, plot = False, extra_len = 1):
    """
    y: np n x 1 array of input data
    """   
    x = np.atleast_2d(np.linspace(0, y.shape[0] - 1, y.shape[0])).T
    
    # squared gaussian kernel
    kernel  = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)

    # initialize gaussian process
    gp = GPy.models.GPRegression(x,y,kernel)
    
    # optimize gp
    gp.optimize()
  
    # point to predict at
    point = (x[-1] + 1).reshape(1, -1)
    
    # predict
    pred, sigma = gp.predict(point)
    
    return_frac = pred / y[-1] - 1
    
    # plotting
    if plot:
        plot_gp(x, y, gp, (point[0,0], pred[0,0]), extra_len)
    
    # return sharpe
    return return_frac[0,0], sigma[0,0]

In [8]:
df = pd.read_csv('/home/nate/Courses/bigdata/bigdata_hw/final_project/SPY_SECOND_TRADE.csv')
X = df['CLOSE'].as_matrix()[:,np.newaxis]

In [9]:
X.shape

(19145, 1)

In [10]:
gauss_gp(X, True)

KeyboardInterrupt: 

In [49]:
def should_buy(X, return_cutoff = 0, sigma_cutoff = 1):
    expected_return, sigma = gauss_gp(X)
    sigma /= X[-1]
    print(expected_return, sigma)
    return expected_return > return_cutoff and sigma < sigma_cutoff
        
        
def trade_period(X, seconds, GP_len = 100):
    returns = []
    trades = 0
    bought = False
    for i in range(seconds):
        buy = should_buy(X[i:GP_len+i])
        if buy ^ bought:
            trades += 1
        if buy:
            returns.append(X[GP_len+i+1] / X[GP_len+i])
        bought = buy
        print(i, 'bought {}'.format(bought))
        
    return returns, trades

In [50]:
returns, trades = trade_period(X, 100, 500)

0.000177562495 [ 0.61512515]
0 bought [ True]
0.000110679074635 [ 0.76392295]
1 bought [ True]
0.000111371838698 [ 0.5961977]
2 bought [ True]
7.65780090708e-05 [ 19.16551735]
3 bought [False]
0.000121837274586 [ 0.57369602]
4 bought [ True]
0.000122661362199 [ 0.57060769]
5 bought [ True]
0.000126888835677 [ 0.57111768]
6 bought [ True]
-0.000274754136202 [ 29.91760746]
7 bought False
-4.61035538801e-06 [ 0.50612947]
8 bought False
-2.34436574631e-06 [ 0.51124594]
9 bought False
1.86305081276e-05 [ 28.08954884]
10 bought [False]
0.000144519306062 [ 0.48937049]
11 bought [ True]
0.000147890764909 [ 0.49126266]
12 bought [ True]
0.000150874592156 [ 0.81759848]
13 bought [ True]
0.000152006548584 [ 0.48582991]
14 bought [ True]
1.19276322881e-05 [ 0.47938955]
15 bought [ True]
0.000157026848362 [ 0.4919089]
16 bought [ True]
0.00015643360556 [ 0.48666495]
17 bought [ True]
4.79477954558e-05 [ 47.76021301]
18 bought [False]
1.77654358617e-05 [ 0.48328137]
19 bought [ True]
2.02951716901e-

In [52]:
np.prod(returns)

0.99957414089059649

In [53]:
trades

42